# **Phân loại tin tức tiếng Việt sử dụng SVM**

In [ ]:
!pip install pyvi

## 1. Xác định bài toán

- Đầu vào bài toán: 1 đoạn tin tức tiếng Việt  
- Đầu ra: Lớp tương ứng với đoạn tin tức đó

- Dataset: VNews8td  
- Mô tả:  Được thu thập từ trang báo mạng VnExpress từ ngày 01/06/2023 - 01/06/2024. Gồm 8 lớp, chia train:val:test theo tỉ lệ 70:10:20. Mỗi văn bản sẽ gồm có tiêu đề và phần mô tả của bài báo  
- 8 lớp: 	
    * doisong (Đời sống)
	* giaoduc (Giáo dục)
	* khoahoc (Khoa học)
	* kinhte (Kinh tế)
	* suckhoe (Sức khỏe)
	* thegioi (Thế giới)
	* thethao (Thể thao)
	* thoisu (Thời sự)

### Import thư viện 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pyvi import ViTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn import svm
from sklearn.metrics import classification_report
from sklearn.metrics import  ConfusionMatrixDisplay

## 2. Tiền xử lý dữ liệu

In [ ]:
train_df = pd.read_csv("/kaggle/input/vnews8td/VNews8td/train.tsv", sep='\t',header=None, names=['văn bản','lớp'])
test_df = pd.read_csv("/kaggle/input/vnews8td/VNews8td/test.tsv", sep='\t',header=None, names=['văn bản','lớp'])
val_df = pd.read_csv("/kaggle/input/vnews8td/VNews8td/val.tsv", sep='\t',header=None, names=['văn bản','lớp'])

In [ ]:
print(train_df.shape)
print(test_df.shape)
print(val_df.shape)

In [ ]:
plt.figure(figsize=(10, 6))
ax = sns.countplot(data=train_df, x='lớp', order=train_df['lớp'].value_counts().index)

plt.title('Số lượng bài báo theo từng lớp (train set)')
plt.xlabel('Lớp')
plt.ylabel('Số lượng')
plt.xticks(rotation=45)

for p in ax.patches:
    height = p.get_height()
    ax.text(p.get_x() + p.get_width()/2, height + 5,  
            f'{int(height)}', ha='center', va='bottom')

plt.tight_layout()
plt.show()

Dữ liệu bị mất cân bằng giữa các lớp --> dùng class weighted, hoặc augment data 

In [ ]:
tokens = ViTokenizer.tokenize(train_df['văn bản'][0]).split()
print(tokens)

In [ ]:
with open('/kaggle/input/vietnamese-stopwords/vietnamese_stopwords.txt', 'r', encoding='utf-8') as f:
    stopwords = list(line.strip() for line in f)

def preprocess(text):
    tokens = ViTokenizer.tokenize(text).split()
    filtered_tokens = [t for t in tokens if t not in stopwords]
    return ' '.join(filtered_tokens)

txt = preprocess(train_df['văn bản'][0])
print(txt)

In [ ]:
print("Danh sách 10 từ dừng đầu tiên (từ không mang ý nghĩa phân loại): ", stopwords[0:10])

Ghép các từ tiếng Việt lại và xóa stop words 

In [ ]:
train_df['processed'] = train_df['văn bản'].apply(preprocess)
val_df['processed'] = val_df['văn bản'].apply(preprocess)
test_df['processed'] = test_df['văn bản'].apply(preprocess)

In [ ]:
train_df['văn bản'][0]

In [ ]:
train_df['processed'][0]

Chuyển dữ liệu từ dạng văn bản về dạng số để xử lý 

In [ ]:
vectorizer = TfidfVectorizer()
X_train = vectorizer.fit_transform(train_df['processed'])
X_val = vectorizer.transform(val_df['processed'])
X_test = vectorizer.transform(test_df['processed'])


In [ ]:
le = LabelEncoder()
y_train = le.fit_transform(train_df['lớp'])
y_val = le.transform(val_df['lớp'])
y_test = le.transform(test_df['lớp'])

In [ ]:
labels = le.classes_
print(labels)

In [ ]:
print("Dữ liệu training: ", X_train.shape, y_train.shape)
print("Dữ liệu validation: ", X_val.shape, y_val.shape)
print("Dữ liệu testing: ", X_test.shape, y_test.shape)

In [ ]:
from sklearn.decomposition import TruncatedSVD
import re

svd = TruncatedSVD(n_components=3, random_state=42)
X_svd = svd.fit_transform(X_train)

plt.figure(figsize=(10, 6))
scatter = plt.scatter(X_svd[:, 0], X_svd[:, 1], c=y_train, cmap='tab10', s=10)

handles, indices = scatter.legend_elements()

legend_labels = [labels[int(re.sub(r"[^\d]", "", i))] for i in indices]
plt.legend(handles, legend_labels, title="Lớp")

plt.title("Trực quan hóa các lớp văn bản")
plt.xlabel("Thành phần 1")
plt.ylabel("Thành phần 2")
plt.grid(True)
plt.tight_layout()
plt.show()

## 3. Huấn luyện mô hình & đánh giá mô hình

*Lưu ý: Đánh giá trên tập validation*

### Kernel linear với C=1

In [ ]:
print("- Training with kernel linear ...")
print("- Train size = {}".format(X_train.shape))
model_linear = svm.SVC(kernel='linear', C=1, class_weight="balanced")
model_linear.fit(X_train, y_train)
print("- model - train complete")


In [ ]:
from sklearn.metrics import confusion_matrix

print("- Testing ...")
y_val_pred = model_linear.predict(X_val)
y_train_pred = model_linear.predict(X_train)
print("\n Classification Report (Train Set):")
print(classification_report(y_train, y_train_pred, target_names=le.classes_))

In [ ]:
print("\n Classification Report (Validation Set):")
print(classification_report(y_val, y_val_pred, target_names=le.classes_))

In [ ]:
print("\n Confusion Matrix (Validation Set):")
cm_val_linear = confusion_matrix(y_val, y_val_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_val_linear, display_labels=labels)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix - SVM - Linear kernel")


### Kernel rbf với C=1

In [ ]:
print("- Training with kernel rbf ...")
print("- Train size = {}".format(X_train.shape))
model_rbf = svm.SVC(kernel='rbf', C=1, class_weight="balanced")
model_rbf.fit(X_train, y_train)
print("- model - train complete")

In [ ]:
print("- Testing with kernel rbf ...")
y_val_pred = model_rbf.predict(X_val)

print("\n Classification Report (Train Set):")
print(classification_report(y_train, y_train_pred, target_names=le.classes_))

In [ ]:
print("\nClassification Report (Validation Set):")
print(classification_report(y_val, y_val_pred, target_names=le.classes_))

In [ ]:
print("\n Confusion Matrix (Validation Set):")
cm_val_rbf = confusion_matrix(y_val, y_val_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_val_rbf, display_labels=labels)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix - SVM - RBF kernel")

### Kernel poly với C=1

In [ ]:
print("- Training with kernel poly ...")
print("- Train size = {}".format(X_train.shape))
model_poly = svm.SVC(kernel='poly', C=1, class_weight="balanced")
model_poly.fit(X_train, y_train)
print("- model - train complete")

In [ ]:
print("- Testing with kernel poly ...")
y_val_pred = model_poly.predict(X_val)
print("\n Classification Report (Train Set):")
print(classification_report(y_train, y_train_pred, target_names=le.classes_))

In [ ]:
print("\nClassification Report (Validation Set):")
print(classification_report(y_val, y_val_pred, target_names=le.classes_))

In [ ]:
print("\n Confusion Matrix (Validation Set):")
cm_val_poly = confusion_matrix(y_val, y_val_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_val_poly, display_labels=labels)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix - SVM - polypoly kernel")

### Kernel sigmoid với C=1

In [ ]:
print("- Training with kernel sigmoid ...")
print("- Train size = {}".format(X_train.shape))
model_sigmoid = svm.SVC(kernel='sigmoid', C=1, class_weight="balanced")
model_sigmoid.fit(X_train, y_train)
print("- model - train complete")


In [ ]:
print("- Testing with kernel sigmoid ...")
y_val_pred = model_sigmoid.predict(X_val)
print("\n Classification Report (Train Set):")
print(classification_report(y_train, y_train_pred, target_names=le.classes_))

In [ ]:
print("\nClassification Report (Validation Set):")
print(classification_report(y_val, y_val_pred, target_names=le.classes_))

In [ ]:
print("\n Confusion Matrix (Validation Set):")
cm_val_sigmoid = confusion_matrix(y_val, y_val_pred)
disp = ConfusionMatrixDisplay(confusion_matrix=cm_val_poly, display_labels=labels)
disp.plot(cmap='Blues', xticks_rotation=45)
plt.title("Confusion Matrix - SVM - Sigmoid kernel")

### Thực hiện sử dụng model đã được train để infer 1 văn bản mới 

In [ ]:
new_doc = "nhập môn trí tuệ nhân tạo"
#Xóa stopwords
new_doc = preprocess(new_doc)
print([new_doc])

input_data_preprocessed = vectorizer.transform([new_doc])
print(input_data_preprocessed)

### Dự đoán nhãn của văn bản trên

In [ ]:
prediction = model_rbf.predict(input_data_preprocessed)
predicted_label = prediction[0]
predicted_category = le.inverse_transform([predicted_label])[0]
print(f"Predicted label index: {predicted_label}")
print(f"Predicted category: {predicted_category}")

## 4. Tối ưu tham số

*Lưu ý: tối ưu tham số trên tập validation, sau đó cuối cùng lấy tham số tốt nhất đánh giá cho tập test* 

In [ ]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from scipy.stats import uniform

# Định nghĩa tham số cần tối ưu
param_dist = [
    {'kernel': ['linear'],'C':[0.1,1,5,10]},
    {'kernel': [ 'rbf', 'sigmoid'],'C': [0.01,0.1, 1, 10],'gamma': [0.01,0.1, 1, 10]}
]
  

# Khởi tạo model SVM
svm = SVC(class_weight='balanced')

# Randomized Search với Cross-Validation
random_search = RandomizedSearchCV(
    estimator=svm, 
    param_distributions=param_dist, 
    n_iter=30,  # Số lần thử nghiệm
    scoring='f1_weighted',
    n_jobs=-1, 
    cv=3, 
    verbose=1,
    random_state=42)
random_search.fit(X_train, y_train)

# Kết quả
print("Best Parameters:", random_search.best_params_)
print("Best Cross-Validation F1 Score:", random_search.best_score_)


In [ ]:
# Đánh giá trên tập test
best_model = random_search.best_estimator_
y_test_pred = best_model.predict(X_test)
print("\nTest Set Classification Report:\n", classification_report(y_test, y_test_pred))

## 5. Cải tiến mô hình

In [ ]:
from sklearn.ensemble import BaggingClassifier

base_svm = SVC(kernel='linear', C=1, class_weight='balanced')
bagging_svm = BaggingClassifier(base_estimator=base_svm, n_estimators=10, max_samples=0.8, random_state=42)

bagging_svm.fit(X_train, y_train)
y_val_pred = bagging_svm.predict(X_val)
y_train_pred = bagging_svm.predict(X_train)
y_test_pred = bagging_svm.predict(X_test)

In [ ]:
print(classification_report(y_train, y_train_pred))

In [ ]:
print(classification_report(y_val, y_val_pred))

In [ ]:
print(classification_report(y_test, y_test_pred))

## 6. Tài liệu tham khảo

- Dataset: https://www.kaggle.com/datasets/dat111/xlnntn-th-lab08
- Vietnamese stopwords: https://github.com/stopwords/vietnamese-stopwords  
- Thư viện preprocessing tiếng Việt: pyvi - https://pypi.org/project/pyvi/